# Persistent Maps

In the previous tutorial, we used HTMap like this:

In [1]:
import htmap

def double(x):
    return 2 * x

In [2]:
mapped = htmap.transient_map(double, range(10))
print(mapped)
doubled = list(mapped)
print(doubled)

<TransientMap(map = <Map(map_id = tmp-1541468851-0)>)>
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In particular, we used the [htmap.transient_map](../api.rst#htmap.transient_map) function to create our map.
This function creates an object that acts almost exactly like the iterator returned by the built-in [map](https://docs.python.org/3/library/functions.html#map) function.
To get our output, we iterated over it using `list`.
Once we've iterated over it, we've exhausted it - the map's data all goes away.

Transient maps are very useful when you want to "fire and forget" - say, for testing that your map works as intended over a small subset of your data.
But to access the full power of HTMap, you'll want to use the persistent mapping functions.

## Creating Persistent Maps

To create a persistent map, we need to give our map a name, which we call a **map ID**:

In [3]:
htmap.remove('dbl')  # we'll get back to this later!
map = htmap.map('dbl', double, range(10))
print(map)

<Map(map_id = dbl)>


The object that was returned by [htmap.map](../api.rst#htmap.map) is a [htmap.Map](../api.rst#htmap.Map).
It gives us a window into the map as it is running, and lets us use the output once the map is finished.

For example, we can print the status of the map:

In [4]:
print(map.status())

Map dbl (10 components): HELD = 0 | IDLE = 10 | RUNNING = 0 | COMPLETED = 0


We can wait for the map to finish:

In [5]:
map.wait(show_progress_bar = True)

dbl: 100%|###################################| 10/10 [00:06<00:00,  4.97input/s]


datetime.timedelta(0, 6, 27635)

Because our map is persistent, we can iterate over it multiple times:

In [6]:
print(list(map))

for d in map:
    print(d)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
0
2
4
6
8
10
12
14
16
18


If we ever lose our reference to it, we can grab a new reference to it using [htmap.load](../api.rst#htmap.load):

In [7]:
new_ref = htmap.load('dbl')

print(new_ref)
print(new_ref is map)  # maps are singletons

<Map(map_id = dbl)>
True


Maps can be recovered from an entirely different Python interpreter session as well.
Suppose you close Python and go on vacation.
You come back and you want to look at your map again, but you've forgotten what you called it.
Just ask HTMap for a list of your map IDs:

In [8]:
print(htmap.map_ids())

('dbl',)


Ok, well, technically it was a tuple, but we'll have to live with it.

HTMap can also print a pretty table showing the status of your maps:

In [9]:
print(htmap.status())

 Map ID │ HELD │ IDLE │ RUNNING │ COMPLETED │   Data  
────────┼──────┼──────┼─────────┼───────────┼─────────
  dbl   │  0   │  0   │    0    │     10    │ 30.3 KB
────────┴──────┴──────┴─────────┴───────────┴─────────


The status message tells us about how many components of our map are in each of the four components states:

* Idle - component is waiting to run
* Running - component is currently executing remotely
* Completed - component is finished executing and output is available
* Held - HTCondor has noticed a problem with the component and is not letting it run

The status of each component of your map is available using the map attribute `component_statuses`:

In [10]:
print(map.component_statuses)

[<ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>, <ComponentStatus.COMPLETED: 'COMPLETED'>]


We'll discuss what to do about held components and how to interact with component statuses in the [Error Handling](error-handling.ipynb) tutorial.

Map IDs are *unique*: if we try to create another map with the same map ID we just used, it will fail:

In [11]:
new_map = htmap.map('dbl', double, range(20))

MapIdAlreadyExists: the requested map_id dbl already exists (recover the Map, then either use or delete it).

As the error message indicates, if we want to re-use the map ID `dbl`, we need to remove the old map first:

In [12]:
map.remove()

[htmap.Map.remove](../api.rst#htmap.Map.remove) deletes all traces of the map.
**It can never be recovered**.
Be careful when using it!

The module-level shortcut [htmap.remove](../api.rst#htmap.remove) lets you skip the intermediate step of getting the actual Map, if you don't already have it.
We used it earlier to make sure you didn't already have a map named `dbl`, which would have raised a `MapIdAlreadyExists` exception back at the start of the tutorial.

Now we can re-use the map ID:

In [13]:
new_map = htmap.map('dbl', double, range(20))
new_map.wait(show_progress_bar = True)
print(list(new_map))

dbl: 100%|###################################| 20/20 [00:12<00:00,  2.84input/s]


[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]


In the [next tutorial](working-with-files.ipynb), we'll see how to tell HTMap to bring a local file along to the execute node.